In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [3]:
spark = (SparkSession.builder
        .master("local[3]")
        .appName("Withcolumn e when")
        .getOrCreate())

In [4]:
df = (spark
          .read
          .format("csv")
          .option("header","true")
          .option("inferSchema","true")
          .option("delimiter",";")
          .load("csv/covid_basico.csv"))

In [5]:
df.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data: string (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- casosAcumulados: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- obitosAcumulados: integer (nullable = true)



In [6]:
df_status_casos_novos = (df.withColumn("Status",
                             F.when(F.col("casosNovos") > 0,
                             F.concat(df.casosNovos, F.lit(" casos novos"))).
                             otherwise("Sem casos novos")).
                             withColumn("Dia", F.substring(F.col("data"),9,2).cast("integer")).
                             withColumn("Mes", F.substring(F.col("data"), 6, 2).cast("integer")).
                             withColumn("Ano", F.substring(F.col("data"), 1, 4).cast("integer"))
                             )

In [7]:
df_status_casos_novos.show(10)

+------+------+----------+----------+---------------+-----------+----------------+---------------+---+---+----+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|         Status|Dia|Mes| Ano|
+------+------+----------+----------+---------------+-----------+----------------+---------------+---+---+----+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|Sem casos novos| 30|  1|2020|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|Sem casos novos| 31|  1|2020|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|Sem casos novos|  1|  2|2020|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|Sem casos novos|  2|  2|2020|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|Sem casos novos|  3|  2|2020|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|Sem casos novos|  4|  

In [8]:
df_status_casos_novos.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data: string (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- casosAcumulados: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- obitosAcumulados: integer (nullable = true)
 |-- Status: string (nullable = true)
 |-- Dia: integer (nullable = true)
 |-- Mes: integer (nullable = true)
 |-- Ano: integer (nullable = true)

